In [ ]:
import pandas as pd

# Đọc dữ liệu từ file Parquet
track_credit_df = pd.read_parquet("data/track_credit_df.par")
tps_df = pd.read_parquet("data/tps.par")

# Hiển thị thông tin cơ bản của dataframe
track_credit_df
tps_df

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

# Thông tin kết nối MySQL
user = "root"
password = "khoa_501"
host = "localhost"
database = "diijam_parquet"

# Tạo engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

try:
    # Tối ưu kích thước cột
    tps_df["listen_count"] = tps_df["listen_count"].astype("int16")
    # Chuyển object thành datetime
    tps_df["date"] = pd.to_datetime(tps_df["date"], errors="coerce") 
    # Thay thế giá trị NULL
    tps_df = tps_df.fillna({"track_id": 0, "user_id": 0, "listen_count": 0})
    
    # Ghi dữ liệu vào MySQL theo từng batch nhỏ
    tps_df.to_sql(
        name="tps",
        con=engine,
        if_exists="replace",
        index=False,
        chunksize=10000  # Ghi từng batch 10.000 dòng
    )
    print("Dữ liệu đã được lưu vào MySQL.")
except SQLAlchemyError as e:
    print(f"Lỗi khi ghi dữ liệu vào MySQL: {e}")
finally:
    engine.dispose()import pandas as pd

In [ ]:
from sqlalchemy.sql import func, select, text

# Thống kê: Đếm số lượng bản ghi
count_query = session.query(func.count()).select_from(TrackCredit).scalar()
print(f"Tổng số bản ghi: {count_query}")

# Tìm giá trị lớn nhất và nhỏ nhất cho một cột (ví dụ cột 'age')
max_query = session.query(func.max(TPS.listen_count)).scalar()
min_query = session.query(func.min(TPS.listen_count)).scalar()
print(f"Giá trị lớn nhất: {max_query}, Giá trị nhỏ nhất: {min_query}")